# Topic Modeling

Topic modeling experiments on TripAdvisor reviews. Dataset provided by the professor of 'CMP617 - Natural Language Processing and Information Recovery' at UFRGS.

## Setup
Installing and importing required libraries.

In [4]:
%pip install bertopic sentence_transformers numpy pandas nltk

from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
  Using cached bertopic-0.16.3-py3-none-any.whl (143 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached umap_learn-0.5.6-py3-none-any.whl (85 kB)
  Using cached hdbscan-0.8.38.post1.tar.gz (5.8 MB)
  distutils: /private/var/folders/jx/d6v3ktf92q96pk6vsyk3pj3r0000gn/T/pip-build-env-gr4e1d_w/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/jx/d6v3ktf92q96pk6vsyk3pj3r0000gn/T/pip-build-env-gr4e1d_w/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/jx/d6v3ktf92q96pk6vsyk3pj3r0000gn/T/pip-build-env-gr4e1d_w/normal'
  distutils: /private/var/folders/jx/d6v3ktf92q96pk6vsyk3pj3r0000gn/T/pip-build-env-gr4e1d_w/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /priv

## Dataset

In [ ]:
DATASET_FILE = 'tripadvisor.csv'
df = pd.read_csv(DATASET_FILE)